<a class = "anchor" id = "top"></a>

# NLP
## Source: Twitter
---
### Authors: Gordon Amoako, Zan Sadiq
---

Table of Contents:
* [Data](#data)
* [Pre-Processing](#eda)
* [Networking](#network)
* [NLP](#nlp)
* [Conclusion](#end)
---

## Data <a class = "anchor" id = "data"></a>

In [13]:
# Import libraries
import pandas as pd
import os
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.types import StructType, StructField, DoubleType, IntegerType, StringType, ArrayType, FloatType
import re
import string
from pyspark.sql.functions import udf, col, size, lit, explode, isnan, when, count
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, IndexToString, StringIndexer, VectorIndexer, CountVectorizer

In [5]:
# Get wd
os.getcwd()

'/home/dataguy/Documents'

In [6]:
# Initialize spark
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
spark = SparkSession.builder.getOrCreate()

22/08/15 07:43:59 WARN Utils: Your hostname, computer resolves to a loopback address: 127.0.1.1; using 192.168.1.159 instead (on interface wlp0s20f3)
22/08/15 07:43:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/15 07:43:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/08/15 07:44:00 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/08/15 07:44:00 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/08/15 07:44:00 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/08/15 07:44:00 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
22/08/15 07:44:00 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
22/08/15 07:44:00 WARN Utils: Service 'SparkUI' could not bind on port 4045. Attempting port 4046.
22/08/15 07:44:00 WARN Utils: Service 'SparkUI' could not bind on port 4046. Attempting port 4047.


In [8]:
df = spark.read.json('/home/dataguy/news.json')

22/08/15 07:45:06 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [15]:
df.printSchema()

root
 |-- _type: string (nullable = true)
 |-- cashtags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- content: string (nullable = true)
 |-- conversationId: long (nullable = true)
 |-- coordinates: struct (nullable = true)
 |    |-- _type: string (nullable = true)
 |    |-- latitude: double (nullable = true)
 |    |-- longitude: double (nullable = true)
 |-- date: string (nullable = true)
 |-- hashtags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- id: long (nullable = true)
 |-- inReplyToTweetId: long (nullable = true)
 |-- inReplyToUser: struct (nullable = true)
 |    |-- _type: string (nullable = true)
 |    |-- created: string (nullable = true)
 |    |-- description: string (nullable = true)
 |    |-- descriptionUrls: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true)
 |    |    |  

In [10]:
df.count()

2770882

In [12]:
df.show(1, False)

+------------------------------+--------+----------------------------------------------------+-------------------+-----------+-------------------------+--------+-------------------+-------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----+---------+-----+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------+-----+----------+-----------+----------------------------------------------------+----------+------------+--------------+------------------------------------------------------------------------------------+-------------------+-----------------------------------+-----------+------------------------------------------------------------+------------

In [14]:
cols = df.columns
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in cols]).show()

AnalysisException: cannot resolve 'isnan(cashtags)' due to data type mismatch: argument 1 requires (double or float) type, however, 'cashtags' is of array<string> type.;
'Aggregate [count(CASE WHEN (isnan(cast(_type#8 as double)) OR isnull(_type#8)) THEN _type END) AS _type#240L, count(CASE WHEN (isnan(cashtags#9) OR isnull(cashtags#9)) THEN cashtags END) AS cashtags#242, count(CASE WHEN (isnan(cast(content#10 as double)) OR isnull(content#10)) THEN content END) AS content#244L, count(CASE WHEN (isnan(cast(conversationId#11L as double)) OR isnull(conversationId#11L)) THEN conversationId END) AS conversationId#246L, count(CASE WHEN (isnan(coordinates#12) OR isnull(coordinates#12)) THEN coordinates END) AS coordinates#248, count(CASE WHEN (isnan(cast(date#13 as double)) OR isnull(date#13)) THEN date END) AS date#250L, count(CASE WHEN (isnan(hashtags#14) OR isnull(hashtags#14)) THEN hashtags END) AS hashtags#252, count(CASE WHEN (isnan(cast(id#15L as double)) OR isnull(id#15L)) THEN id EN

[Back to top...](#top)

## Pre-Processing <a class = "anchor" id = "eda"></a>

[Back to top...](#top)

## Networking <a class = "anchor" id = "network"></a>

[Back to top...](#top)

## NLP: <a class = "anchor" id = "nlp"></a>

[Back to top...](#top)

## Conclusion: <a class = "anchor" id = "end"></a>

[Back to top...](#top)